In [5]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from copy import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import fnmatch
import csv
import pickle
import warnings
import torch
import scipy
from scipy import optimize
from scipy.optimize import leastsq
from io import StringIO
from scipy.signal import savgol_filter

path = r'Data/Results' # use your path
all_files = glob.glob(path + "/*33(a).csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [6]:
pseudo_x = frame.iloc[:,0]
pseudo_y = frame.iloc[:,1]
pseudo_pred = frame.iloc[:,2]

In [9]:
%run Functions.ipynb
nbins = 40

def get_mean_pseudodata(y):
    df_train, cuts1, cuts2 = ewd(pseudo_x, y)
    mean, std, count = binned_statistics(pseudo_x,y)
    #meanvector = vectorize_mean(pseudo_x,y)
    return mean, std


    

In [10]:
pseudo_mean, pseudo_std = get_mean_pseudodata(pseudo_pred)
pseudo_meanvector = np.linspace(-1, 1, nbins)